In [ ]:
import math

# Schreibmaschinendistanz

Gewichtung der Tippfehler-Distanzen anhand des sprachspezifischen Tastatur-Layouts. Für Rechtschreibfehlererkennung sind nur Zeichen des Alphabets relevant, vom Prinzip her könnte dieser Algorithmus jedoch auf alle Zeichen einer Tastatur erweitert werden.

In [ ]:
# Tastatur-Layout für deutsch/östereichische Tastaturen
# Für andere Sprachen ist das Tastatur-Layout entsprechend zu definieren.

# Jeder Schlüssel des Dictionaries repräsentiert eine Taste auf der Tastatur.
# Die direkt angrenzenden Tasten werden als Liste repräsentiert.
layoutDE = {
    'a':['q','s','w','y'],
    'b':['g','h','n','v'],
    'c':['x','d','f','v'],
    'd':['s','e','r','f','c','x'],
    'e':['w','r','d','s'],
    'f':['d','r','t','g','v','c'],
    'g':['f','t','z','h','b','v'],
    'h':['g','z','u','j','n','b'],
    'i':['u','o','k','j'],
    'j':['h','u','i','k','m','n'],
    'k':['j','i','o','l','m'],
    'l':['k','o','p','ö'],
    'm':['n','j','k'],
    'n':['b','h','j','m'],
    'o':['i','p','l','k'],
    'p':['o','ß','ü','ö','l'],
    'q':['w','a'],
    'r':['e','t','f','d'],
    's':['a','w','e','d','x','y'],
    't':['r','z','g','f'],
    'u':['z','i','j','h'],
    'v':['c','f','g','b'],
    'w':['q','e','s','a'],
    'v':['c','f','g','b'],
    'x':['y','s','d','c'],
    'y':['a','s','x'],
    'z':['t','u','h','g'],
    'ä':['ö','ü'],
    'ö':['l','p','ü','ä'],
    'ü':['p','ß','ä','ö'],
    'ß':['ü','p']
}

In [ ]:
class typewriter_distance:
    
    def __init__(self, keyboardLayoutGraph):
        """
            :type keyboardLayoutGraph: dictionary with keyboard characters as keys, 
                                       values are lists of their surrounding characters
        """
        self._layout = keyboardLayoutGraph
        self._knoten = list(self._layout.keys()) 
        self._kanten = [] # Liste von 3-Tupeln (knoten1, knoten2, kosten)
    
        for k1 in self._knoten:
            self._kanten.extend([(k1,k2,1) for k2 in self._layout[k1]])

    def shortestPath(self, start, ziel):
        # Source: https://de.wikibooks.org/wiki/Algorithmensammlung:_Graphentheorie:_Dijkstra-Algorithmus 
        # Adaptierte Implementierung von Dijkstra's Algorithmus
        #
        # start ist der Knoten, in dem die Suche startet
        # ziel ist der Knoten, zu dem ein Weg gesucht werden soll
        # Gibt ein Tupel zurück mit dem Weg und den Kosten 
        #
        knotenEigenschaften = [ [i, float('inf'), None, False] for i in self._knoten if i != start ]
        knotenEigenschaften += [ [start, 0, None, False] ]
        for i in range(len(knotenEigenschaften)):
            knotenEigenschaften[i] += [ i ]

        while True:
            unbesuchteKnotenIterator = filter(lambda x: not x[3], knotenEigenschaften)
            unbesuchteKnoten=list(unbesuchteKnotenIterator)
            if not unbesuchteKnoten:
                break
            sortierteListe = sorted(unbesuchteKnoten, key=lambda i: i[1])
            # knoten mit geringsten Kosten als besucht markieren
            aktiverKnoten = sortierteListe[0]
            knotenEigenschaften[aktiverKnoten[4]][3] = True
            if aktiverKnoten[0] == ziel:
                # zielknoten erreicht, terminieren
                break
            # von aktivem Knoten ausgehende Kanten ermitteln    
            aktiveKanten = list(filter(lambda x: x[0] == aktiverKnoten[0], self._kanten))
            for kante in aktiveKanten:
                # zielknoten liste ermitteln
                andereKnotenListe=list(filter(lambda x: x[0] == kante[1], knotenEigenschaften))
                andererKnotenId=andereKnotenListe[0][4]
                gewichtSumme = aktiverKnoten[1] + kante[2]
                if gewichtSumme < knotenEigenschaften[andererKnotenId][1]:
                    # kürzeren Weg zum zielknoten gefunden
                    knotenEigenschaften[andererKnotenId][1] = gewichtSumme
                    knotenEigenschaften[andererKnotenId][2] = aktiverKnoten[4]


        if aktiverKnoten[0] == ziel:
            weg = []
            weg += [ aktiverKnoten[0] ]

            kosten = aktiverKnoten[1]
            while aktiverKnoten[0] != start:
                # pfad des kürzesten Weges ermitteln
                aktiverKnoten = knotenEigenschaften[aktiverKnoten[2]]
                weg += [ aktiverKnoten[0] ]

            weg.reverse()
            return (weg, kosten)
        else:
            raise Exception("Kein Weg gefunden zwischen",str(start)," und ",ziel)
            
    def distance(self,x,y):
        # Ermittelt die Typewriter Distanz zwischen zwei Buchstaben
        return self.shortestPath(x,y)[1]
    
    def weight (self,c1,c2):
        # Gewichtung anhand der Entfernung zweier Buchstaben
        return 1-1/(2**self.distance(c1,c2))

In [ ]:
twd = typewriter_distance(layoutDE)

In [ ]:
# kürzester Pfad zwischen zwei Tasten

print(twd.shortestPath('s', 'f'))
print(twd.shortestPath('s', 'r'))
print(twd.shortestPath('p', 's'))
print(twd.shortestPath('y', 'ß'))

In [ ]:
def weight (c1,c2):
    return 1-1/(2**twd.distance(c1,c2))

In [ ]:
print(twd.weight('s', 'd'))
print(twd.weight('s', 'r'))
print(twd.weight('p', 's'))
print(twd.weight('y', 'ß'))

# Modifizierte Levenshtein Distanz

Substituierte Zeichen werden anhand ihrer Typewriter Distance gewichtet. 

In [ ]:
# basierend auf https://en.wikibooks.org/wiki/Algorithm_Implementation/Strings/Levenshtein_distance
"""
Compute the Levenshtein distance between two given strings (s1 and s2). 
Cost of substitutions are weighted by the  distance of the subsituted character on a keyboard. 
"""
def levenshtein_typewriter_distance(s1, s2):
    if len(s1) < len(s2):
        return levenshtein_typewriter_distance(s2, s1)

    # len(s1) >= len(s2)
    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 # j+1 instead of j since previous_row and current_row are one character longer
            deletions = current_row[j] + 1       # than s2
            cost = 0 if c1 == c2 else twd.weight(c1,c2)
            substitutions = previous_row[j] + cost
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]

In [ ]:
levenshtein_typewriter_distance('fliegen','floegen')

# Modifizierte Damerau-Levenshtein Distanz

Substituierte Zeichen werden anhand ihrer Typewriter Distance gewichtet. 

In [ ]:
# basierend auf https://www.guyrutenberg.com/2008/12/15/damerau-levenshtein-distance-in-python/
"""
Compute the Damerau-Levenshtein distance between two given strings (s1 and s2). 
Cost of substitutions are weighted by the  distance of the subsituted character on a keyboard. 
"""
def damerau_levenshtein_typewriter_distance(a, b):
    # "Infinity" -- greater than maximum possible edit distance
    # Used to prevent transpositions for first characters
    INF = len(a) + len(b)

    # Matrix: (M + 2) x (N + 2)
    matrix  = [[INF for n in range(len(b) + 2)]]
    matrix += [[INF] + [n for n in range(len(b) + 1)]]
    matrix += [[INF, m] + [0] * len(b) for m in range(1, len(a) + 1)]

    # Holds last row each element was encountered: DA in the Wikipedia pseudocode
    last_row = {}

    # Fill in costs
    for row in range(1, len(a) + 1):
        # Current character in a
        ch_a = a[row-1]

        # Column of last match on this row: DB in pseudocode
        last_match_col = 0

        for col in range(1, len(b) + 1):
            # Current character in b
            ch_b = b[col-1]

            # Last row with matching character
            last_matching_row = last_row.get(ch_b, 0)

            # Cost of substitution
            cost = 0 if ch_a == ch_b else twd.weight(ch_a,ch_b)

            # Compute substring distance
            matrix[row+1][col+1] = min(
                matrix[row][col] + cost, # Substitution
                matrix[row+1][col] + 1,  # Addition
                matrix[row][col+1] + 1,  # Deletion

                # Transposition
                # Start by reverting to cost before transposition
                matrix[last_matching_row][last_match_col]
                    # Cost of letters between transposed letters
                    # 1 addition + 1 deletion = 1 substitution
                    + max((row - last_matching_row - 1),
                          (col - last_match_col - 1))
                    # Cost of the transposition itself
                    + 1)

            # If there was a match, update last_match_col
            if cost == 0:
                last_match_col = col

        # Update last row for current character
        last_row[ch_a] = row

    # Return last element
    return matrix[-1][-1]

In [ ]:
damerau_levenshtein_typewriter_distance('fliegen','floegen')